# LU分解法のベンチマーク

同じ係数行列を持つ複数の連立１次方程式
$$ 
Ax_i = b_i  (i=1,2,...,N)
$$

に対して，LU分解法はGaussの消去法を繰り返すよりも
理論的には速いといわれる．どれくらい速くなるか考察しつつ，ベンチマークで検証する．

## Gaussの消去法 
$A\vec{x} = \vec{b}$ をGaussの消去法で
解いたときに要する時間を `BenchmarkTools` の
`@btime`マクロで計測する．

In [10]:
#]add BenchmarkTools  # パッケージのインストール必要

In [19]:
using BenchmarkTools  
A = rand(100,100); 
b1 = rand(100);
b2 = rand(100);
b3 = rand(100);

@btime A\b1;
@btime (A\b1; A\b2; A\b3;);

  119.791 μs (4 allocations: 79.92 KiB)
  362.125 μs (12 allocations: 239.77 KiB)


実行時間は1本あたり，およそ 120[μs]となった．
したがって，$N$本の場合は 

**120N**[μs]

と推定される．

## LU分解
LU分解を1度計算し，

In [20]:
using LinearAlgebra
@btime L,U = lu(A);
@btime L,U = lu(A,NoPivot());   # pivoting無しの場合

  131.458 μs (10 allocations: 235.45 KiB)
  149.375 μs (10 allocations: 235.45 KiB)


LU分解についても時間を計測すると，約130[μs]となった．

比較のためにpivotingを無しにして計測すると．約150[μs]と若干遅くなった．

LU分解法により解を計算すると，

In [24]:
L,U = lu(A)
@btime U\(L\b1);   # 1本の場合
@btime (U\(L\b1); U\(L\b2); U\(L\b3));   # 3本の場合

  9.417 μs (2 allocations: 1.75 KiB)
  28.375 μs (6 allocations: 5.25 KiB)


実行時間は1本あたり約 9[μs]となった．
LU分解にかかる時間が 130 [μs] だから，
$N$本の場合は **130+ 9N** [μs] となる．

ここで，`T` が 上三角 or 下三角行列の場合，`T\b`は後退 or 前進代入により解が計算されることに注意
・・・【[参考](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Base.:\\-Tuple{AbstractMatrix,%20AbstractVecOrMat})】
    
## ノート
右辺のベクトル$\vec{b}^{(1)}, \ldots, \vec{b}^{(N)}$が一度に与えられている場合は，
次のようにしたほうが最も速く計算できるだろう．

In [ ]:
@btime A\[b1 b2 b3]